In [1]:
import pandas, seaborn, scipy, numpy, matplotlib, collections, itertools, math, functools, sys, sklearn, os

/Users/tim/miniconda3/envs/py3k/lib/python3.5/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


# Sources (samples)

In [2]:
ega_filenames = pandas.read_table("../data/external/ega_contents.aspera.tsv", header=None)[0]
ega_filenames = pandas.Series([x.replace(".aes", "") for x in ega_filenames])

downloaded_file_paths = pandas.read_table("../data/external/downloaded_file_paths.txt", header=None)[0]

ISSUE_EGA_FILENAMES = set()
def lookup_filename(substring):
    results = downloaded_file_paths[downloaded_file_paths.str.contains(substring)]
    ega_results = set(ega_filenames[ega_filenames.str.contains(substring)])
    
    if len(results) > 1:
        results = [x for x in results if next(iter(ega_results)) in x]

    if len(results) == 0:
        print("NOT_FOUND: %s, EGA results: %s" % (substring, ega_results))
        ISSUE_EGA_FILENAMES.update(ega_results)
        return "NOT_FOUND:%s" % substring
    if len(results) > 1:
        print("MULTIPLE_FOUND: %s %s, EGA results: %s" % (substring, sorted(results), ega_results))
        #ISSUE_EGA_FILENAMES.update(ega_results)
        return "MULTIPLE_FOUND:%s" % substring
    return list(results)[0]


In [3]:
samples = pandas.read_table("../data/external/sample.tsv")
specimens = pandas.read_table("../data/external/specimen.tsv")
ids = pandas.read_csv("../data/external/ICGC_IDs_19Oct2015_External_modified.csv")

sources = pandas.merge(ids, samples, left_on='DNA_biospecimen', right_on="submitted_sample_id", how='inner')
sources = pandas.merge(sources, specimens, on="icgc_specimen_id", how='inner').copy()


assert all(sources.submitted_donor_id_x == sources.submitted_donor_id_y)
assert all(sources.icgc_donor_id_x == sources.icgc_donor_id_y)

sources["cohort"] = "AOCS"
sources["donor"] = sources.submitted_donor_id_x
sources["tissue_type"] = sources.SpecimenType.map({"Tumour": "solid", "Ascites": "ascites"})
sources["timepoint"] = sources.CollectionPoint.map(
    {"Primary": "primary", "Recurrence": "recurrence", "Autopsy": "recurrence"})

sources["source_id"] = sources.DNA_biospecimen
sources.index = sources.source_id

sources["treated"] = sources.specimen_donor_treatment_type == "other therapy"
sources["metastasis"] = sources.specimen_type_description.str.contains("metastasis")
sources["interval_days"] = sources.specimen_interval

print('looking for dna')
sources["bam_path_tumor_dna"] = [lookup_filename(row.DNA_sample_string) for (i,row) in sources.iterrows()]

print("looking for rna")
sources["bam_path_tumor_rna"] = [lookup_filename(row["RNA data file"]) for (i,row) in sources.iterrows()]

sources.loc["AOCS-170-1-8", "treated"] = False  # weird incorrect data point based on email from Elizabeth Christie

assert sources["source_id"].nunique() == 114
assert len(sources["source_id"]) == 114

simple_sources = sources[
    "source_id,donor,cohort,treated,timepoint,metastasis,tissue_type,interval_days,bam_path_tumor_dna,bam_path_tumor_rna".split(",")
]

sources.to_csv("../data/derived/sources.full.csv", index=False)
simple_sources.to_csv("../data/derived/sources.simple.csv", index=False)


looking for dna
looking for rna


# Mutations

In [4]:
# Load Patch et al mutation calls.
aocs_all_ssm_df = pandas.read_table("../data/external/ssm_open.tsv.gz")

In [5]:
aocs_all_ssm_df

,icgc_mutation_id,icgc_donor_id,project_code,icgc_specimen_id,icgc_sample_id,matched_icgc_sample_id,submitted_sample_id,submitted_matched_sample_id,chromosome,chromosome_start,...,experimental_protocol,sequencing_strategy,base_calling_algorithm,alignment_algorithm,variation_calling_algorithm,other_analysis_algorithm,seq_coverage,raw_data_repository,raw_data_accession,initial_data_release_date
0,MU8743702,DO46493,OV-AU,SP101906,SA505498,SA505512,AOCS-139-19-0,AOCS-139-5-X,18,61407798,...,Paired End http://www.illumina.com/technology/...,WGS,RTA http://support.illumina.com/sequencing/seq...,bwa http://bio-bwa.sourceforge.net/,qSNP http://qcmg.org/informatics.php,GATK http://www.broadinstitute.org/gatk/,NaN,EGA,EGAS00001000154,NaN
1,MU8750236,DO46493,OV-AU,SP101906,SA505498,SA505512,AOCS-139-19-0,AOCS-139-5-X,6,63262673,...,Paired End http://www.illumina.com/technology/...,WGS,RTA http://support.illumina.com/sequencing/seq...,bwa http://bio-bwa.sourceforge.net/,qSNP http://qcmg.org/informatics.php,GATK http://www.broadinstitute.org/gatk/,NaN,EGA,EGAS00001000154,NaN
2,MU8750236,DO46493,OV-AU,SP101906,SA505498,SA505512,AOCS-139-19-0,AOCS-139-5-X,6,63262673,...,Paired End http://www.illumina.com/technology/...,WGS,RTA http://support.illumina.com/sequencing/seq...,bwa http://bio-bwa.sourceforge.net/,qSNP http://qcmg.org/informatics.php,GATK http://www.broadinstitute.org/gatk/,NaN,EGA,EGAS00001000154,NaN
3,MU8751505,DO46493,OV-AU,SP101906,SA505498,SA505512,AOCS-139-19-0,AOCS-139-5-X,18,63487174,...,Paired End http://www.illumina.com/technology/...,WGS,RTA http://support.illumina.com/sequencing/seq...,bwa http://bio-bwa.sourceforge.net/,qSNP http://qcmg.org/informatics.php,GATK http://www.broadinstitute.org/gatk/,NaN,EGA,EGAS00001000154,NaN
4,MU8751505,DO46493,OV-AU,SP101906,SA505498,SA505512,AOCS-139-19-0,AOCS-139-5-X,18,63487174,...,Paired End http://www.illumina.com/technology/...,WGS,RTA http://support.illumina.com/sequencing/seq...,bwa http://bio-bwa.sourceforge.net/,qSNP http://qcmg.org/informatics.php,GATK http://www.broadinstitute.org/gatk/,NaN,EGA,EGAS00001000154,NaN
5,MU8751505,DO46493,OV-AU,SP101906,SA505498,SA505512,AOCS-139-19-0,AOCS-139-5-X,18,63487174,...,Paired End http://www.illumina.com/technology/...,WGS,RTA http://support.illumina.com/sequencing/seq...,bwa http://bio-bwa.sourceforge.net/,qSNP http://qcmg.org/informatics.php,GATK http://www.broadinstitute.org/gatk/,NaN,EGA,EGAS00001000154,NaN
6,MU8768048,DO46493,OV-AU,SP101906,SA505498,SA505512,AOCS-139-19-0,AOCS-139-5-X,18,68110673,...,Paired End http://www.illumina.com/technology/...,WGS,RTA http://support.illumina.com/sequencing/seq...,bwa http://bio-bwa.sourceforge.net/,qSNP http://qcmg.org/informatics.php,GATK http://www.broadinstitute.org/gatk/,NaN,EGA,EGAS00001000154,NaN
7,MU8768921,DO46493,OV-AU,SP101896,SA505517,SA505512,AOCS-139-6-3,AOCS-139-5-X,18,68385067,...,Paired End http://www.illumina.com/technology/...,WGS,RTA http://support.illumina.com/sequencing/seq...,bwa http://bio-bwa.sourceforge.net/,qSNP http://qcmg.org/informatics.php,GATK http://www.broadinstitute.org/gatk/,NaN,EGA,EGAS00001000154,NaN
8,MU8768921,DO46493,OV-AU,SP101906,SA505498,SA505512,AOCS-139-19-0,AOCS-139-5-X,18,68385067,...,Paired End http://www.illumina.com/technology/...,WGS,RTA http://support.illumina.com/sequencing/seq...,bwa http://bio-bwa.sourceforge.net/,qSNP http://qcmg.org/informatics.php,GATK http://www.broadinstitute.org/gatk/,NaN,EGA,EGAS00001000154,NaN
9,MU8776724,DO46493,OV-AU,SP101906,SA505498,SA505512,AOCS-139-19-0,AOCS-139-5-X,6,68017479,...,Paired End http://www.illumina.com/technology/...,WGS,RTA http://support.illumina.com/sequencing/seq...,bwa http://bio-bwa.sourceforge.net/,qSNP http://qcmg.org/informatics.php,GATK http://www.broadinstitute.org/gatk/,NaN,EGA,EGAS00001000154,NaN


In [6]:
donor_to_sources = collections.defaultdict(set)
for (i, row) in sources.iterrows():
    donor_to_sources[row.donor].add(row.source_id)

icgc_specimen_id_to_source_id = dict((v,k) for (k,v) in sources.icgc_specimen_id.to_dict().items())

mutations_df = aocs_all_ssm_df.copy()
mutations_df["donor"] = ["-".join(x.split("-")[:2]) for x in mutations_df.submitted_sample_id]
mutations_df["source_id"] = mutations_df.icgc_specimen_id.map(icgc_specimen_id_to_source_id)

# Drop Y chromosome mutations
mutations_df = mutations_df[mutations_df.chromosome != "Y"].drop_duplicates()

# rename columns for varlens compatability
mutations_df["genome"] = "GRCh37"
mutations_df["contig"] = mutations_df["chromosome"]
mutations_df["interbase_start"] = mutations_df["chromosome_start"] - 1
mutations_df["interbase_end"] = mutations_df["chromosome_end"]
mutations_df["ref"] = mutations_df["mutated_from_allele"]
mutations_df["alt"] = mutations_df["mutated_to_allele"]
del mutations_df["chromosome"]
del mutations_df["chromosome_start"]
del mutations_df["chromosome_end"]
del mutations_df["mutated_from_allele"]
del mutations_df["mutated_to_allele"]

# duplicate rows so each mutation that occurs in a donor is given for every source for that donor
to_concat = []
for (donor, sub_df) in mutations_df.groupby("donor"):
    for source_id in donor_to_sources[donor]:
        sub_df = sub_df.copy()
        sub_df["source_id"] = source_id
        to_concat.append(sub_df)
expanded_mutations_df = pandas.concat(to_concat, ignore_index=True)
print("Expanded %d -> %d" % (len(mutations_df), len(expanded_mutations_df)))

# connect adjacent variants into dinucleotide variants
multinucleotide_additions = []
multinucleotide_indices = []
for (source, sub_df) in expanded_mutations_df.groupby("source_id"):
    #print(source, len(multinucleotide_additions))
    site_to_row = dict(((row.contig, row.interbase_start), (i,row)) for (i,row) in sub_df.iterrows() )
    for (i,row) in sub_df.iterrows():
        if (row.contig, row.interbase_start + 1) in site_to_row and (row.contig, row.interbase_start - 1) not in site_to_row:
            accumulated = []
            while True:                
                try:
                    accumulated.append(site_to_row[(row.contig, row.interbase_start + len(accumulated))])
                except KeyError:
                    break
            assert len(accumulated) > 1
            multinucleotide_indices.extend(x for (x, _) in accumulated)
            new_row = accumulated[0][1].copy()
            new_row.interbase_end = accumulated[-1][1].interbase_end
            new_row.ref = "".join([x[1].ref for x in accumulated])
            new_row.alt = "".join([x[1].alt for x in accumulated])
            multinucleotide_additions.append(new_row)
                       
multinucleotides_df = pandas.DataFrame(multinucleotide_additions)
print("Multinucleotides: %d" % len(multinucleotides_df))

expanded_mutations_non_mnv_df = expanded_mutations_df.ix[~expanded_mutations_df.index.isin(multinucleotide_indices)]
combined_df = pandas.concat([expanded_mutations_non_mnv_df, multinucleotides_df], ignore_index=True)
print(combined_df.shape, expanded_mutations_non_mnv_df.shape, multinucleotides_df.shape, mutations_df.shape)


Expanded 3288030 -> 5324182
Multinucleotides: 53850
(5349387, 45) (5295537, 45) (53850, 45) (3288030, 45)


In [7]:
columns = "source_id,donor,genome,contig,interbase_start,interbase_end,ref,alt".split(",")
combined_df[columns].to_csv("../data/derived/raw_mutations.csv", index=False)
!bzip2 ../data/derived/raw_mutations.csv